<a href="https://colab.research.google.com/github/madarshb19/CNNs-for-Image-Classification/blob/main/CNNsForCatsAndDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator   #helps in preprocessing images

In [ ]:
tf.__version__

'2.9.2'

## Part 1 - Data Preprocessing

Since the dataset consists of images,data preprocessing would look different

### Preprocessing the Training set

In [ ]:
#we apply transformation on training sets to avoid overfitting (this is also done in a general computervision problem)
#this is called image augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,  #applies feature scaling to every pixel
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64), #size of images (changed from 150 to 64 for faster computation)
        batch_size=32,
        class_mode='binary')

### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) #transformation is not done on test set,only feature scaling is performed
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3,activation = 'relu',input_shape = [64,64,3])) #number of filters can be experimented,kernel size/feature detector = 3*3 which is most used,input shape = 64*64 layer. for rgb 64 64 3,for b/w 64 64 1

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2,strides = 2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3,activation = 'relu')) #input shape parameter is only to connect the first input layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2,strides = 2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units= 128,activation = 'relu')) #no of units can also be experimented

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units= 1,activation = 'sigmoid')) #for binary,units = 1

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set,validation_data = test_set,batch_size = 32,epochs = 25)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img(
    'dataset/single_prediction/cat_or_dog_1.jpg',
    target_size = (64,64)   #converting given image to 64*64 for training
)
test_image = image.img_to_array(test_image)
#while training the network,there was an additional dimension of batch size = 32 so we need to add this batch dimension below
test_image = np.expand_dims(test_image,axis = 0)
result = cnn.predict(test_image) #result will return 0 or 1,now we have to find which is 0 and which is 1
training_set.class_indices #this gives the corresponding binary class
if(result[0][0] == 1):
  prediction = 'Prediction : Dog'
else:
  prediction = 'Prediction : Cat'

In [ ]:
print(prediction)